In [11]:
#import ctranslate2
#ctranslate2.converters.TransformersConverter(model_name_or_path='openai/whisper-large-v3',copy_files=['tokenizer.json', 'preprocessor_config.json']).convert(output_dir='whisper-large-v3-ct2', quantization='float16')

In [12]:
from collections import Counter
from faster_whisper import WhisperModel
import json
import os

In [13]:
contexto = "Essa é uma transcrição de uma ligação para avialiação de NPS da empresa TOTVS."
audio_path="audio_samples/2874774.wav"

In [14]:
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [15]:
# Transcrição do áudio
segments, info = model.transcribe(audio=audio_path,
                                  language='pt',
                                  beam_size=5,
                                  initial_prompt=contexto
                                  )

In [16]:
# Inicializando variáveis para métricas
total_words = 0
word_count = Counter()
magic_words = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe", 
               "por gentileza", "bom dia", "boa tarde", "boa noite", 
               "agradeço", "agradece", "gratidão", "sinto muito", 
               "perdão", "me perdoe", "com licença"]
magic_word_count = Counter({word: 0 for word in magic_words})
total_duration = 0

In [17]:
# Updated transcription and metrics calculation
transcription_data_optimized = {
    "prompt": contexto,
    "model": model_size,
    "audio_path": audio_path,
    "transcription": [],  # This needs to be filled with the actual transcriptions
    "metrics": {}
}

In [18]:
# Processando a transcrição para calcular métricas
id = 0

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

    # Dividindo o texto em palavras e convertendo para minúsculas
    words = segment.text.lower().split()
    total_words += len(words)
    word_count.update(words)

    # Contabilizando palavras mágicas
    for word in magic_words:
        magic_word_count[word] += words.count(word)

    # Calculando duração total do áudio em segundos
    duration = (segment.end - segment.start)
    total_duration += (segment.end - segment.start)
    
    temp = {
        "order": id,
        "start": segment.start,
        "end": segment.end,
        "duration": duration,
        "transcription": segment.text
        }
    
    # Aqui é onde você adiciona o temp à lista "transcription"
    transcription_data_optimized["transcription"].append(temp)
    
    id += 1

[0.00s -> 7.06s]  Bom dia, Natália. Beatriz da TOTV falando. Gostaria de falar que a senhora Silvânia é do TI.
[7.06s -> 11.52s]  Seria ela ainda responsável pelo sistema? Só um momento. Obrigada.
[23.90s -> 27.78s]  Silvânia. Bom dia, senhora Silvânia. Beatriz da TOTV falando. Tudo bem?
[27.78s -> 30.52s]  Bom dia, Beatriz. Tudo bom? E você?
[31.10s -> 36.94s]  Também obrigada. A senhora Silvânia, a senhora seria responsável pelo sistema na empresa Escandinávia Veículos, né?
[38.90s -> 39.30s]  Sim.
[39.70s -> 44.04s]  Tá. O meu contato é bem rápido. Eu falo da parte da experiência do cliente.
[44.12s -> 49.00s]  Eu gostaria de saber somente como que está aí a experiência, acompanhar um pouco a percepção que vocês estão tendo.
[49.50s -> 54.10s]  Esse acompanhamento é feito pelo método de NPS. Onde a senhora avalia de 0 a 10 da marca TOTVS?
[54.10s -> 56.62s]  Alguns pontos específicos. Dois minutinhos, tudo bem?
[56.62s -> 59.70s]  A senhora seria responsável pelo TI, né?
[62.18s -> 

In [19]:
# Efficient calculations and metrics assignment
if total_duration > 0 and total_words > 0:
    words_per_minute = (total_words / total_duration) * 60
    sorted_word_count = word_count.most_common(10)  # Only compute the top 10 words
    magic_word_percentages = {word: (count / total_words) * 100 for word, count in magic_word_count.items()}

    # Assign computed metrics to the data dictionary
    transcription_data_optimized["metrics"].update({
        "total_words": total_words,
        "words_per_minute": words_per_minute,
        "top_10_words": sorted_word_count,
        "magic_word_percentages": magic_word_percentages,
    })
else:
    transcription_data_optimized["metrics"].update({
        "total_words": total_words,
        "words_per_minute": 0,
        "top_10_words": [],
        "magic_word_percentages": {word: 0 for word in magic_words},
    })


# Exibindo as métricas
print("Métricas:")
print(f"\nQuantidade total de palavras: {total_words}")

print(f"\nQuantidade de palavras por minuto: {words_per_minute:.2f}")

print("\nTop 10 palavras mais faladas:")
for word, count in sorted_word_count[:10]:
    print(f"{word}: {count}")

print("\nPercentual de palavras mágicas:")
for word, percentage in magic_word_percentages.items():
    print(f"{word}: {percentage:.2f}%")

Métricas:

Quantidade total de palavras: 655

Quantidade de palavras por minuto: 168.03

Top 10 palavras mais faladas:
a: 51
senhora: 23
o: 19
que: 18
de: 14
do: 10
da: 9
é: 9
tá.: 9
eu: 8

Percentual de palavras mágicas:
obrigado: 0.00%
obrigada: 0.00%
por favor: 0.00%
desculpa: 0.00%
desculpe: 0.00%
por gentileza: 0.00%
bom dia: 0.00%
boa tarde: 0.00%
boa noite: 0.00%
agradeço: 0.15%
agradece: 0.00%
gratidão: 0.00%
sinto muito: 0.00%
perdão: 0.00%
me perdoe: 0.00%
com licença: 0.00%


In [20]:
# Enhanced JSON writing with error handling
json_path = 'json_files'
os.makedirs(json_path, exist_ok=True)

json_file = os.path.join(json_path, os.path.basename(os.path.splitext(audio_path)[0])) + '.json'

try:
    with open(json_file, "w", encoding="utf-8") as f:
        json.dump(transcription_data_optimized, f, ensure_ascii=False, indent=4)
    print(f"Transcription and metrics successfully saved in {json_file}")
except IOError as e:
    print(f"Failed to save JSON file: {e}")

Transcription and metrics successfully saved in json_files/2874774.json
